# AI cập nhật dữ liệu mới nhất cho báo cáo tuần

In [24]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import import_gemini
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload
import other_source_data

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(import_gemini)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)
importlib.reload(other_source_data)

from import_default import *
from import_database import *
from import_other import *
from import_gemini import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *
from other_source_data import *

- Lấy dữ liệu từ Mongo

In [25]:
date_series = get_mongo_collection(ref_db, 'date_series')
time_series = get_mongo_collection(ref_db, 'time_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')

if date_series['date'].max() < get_today_date():
    new_date = get_today_date()
    date_series = pd.concat([date_series, pd.DataFrame({'date': [new_date]})]).sort_values(by='date', ascending=False).reset_index(drop=True)

In [26]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "pct_change": 1,
    "diff": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "year_open": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
	"YFIBO_0382": 1,
    "YFIBO_0500": 1,
    "YFIBO_0618": 1,
}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

In [27]:
projection = {"_id": 0,"date": 1,"ticker": 1,"trend_5p": 1,"trend_20p": 1,"trend_60p": 1,"trend_240p": 1}
today_ms_chart_df = get_mongo_collection(stock_db, "today_ms_chart", find_query={"ticker":'all'}, projection=projection)
history_ms_chart_df = get_mongo_collection(stock_db, "history_ms_chart", find_query={"ticker":'all'}, projection=projection)
full_ms_chart_df = pd.concat([today_ms_chart_df, history_ms_chart_df], axis=0, ignore_index=True).drop(columns=['ticker'])

today_stock_df = get_mongo_collection(stock_db, "today_stock", find_query={"date": {"$in": date_series['date'].iloc[:5].tolist()}})
history_stock_df = get_mongo_collection(stock_db, "history_stock", find_query={"date": {"$in": date_series['date'].iloc[:5].tolist()}})
origin_stock_df = pd.concat([today_stock_df, history_stock_df], axis=0, ignore_index=True)
full_stock_df = origin_stock_df[["date","ticker","open","high","low","close","volume",'cap','t5_score']]

other_ticker_df = get_mongo_collection(stock_db, 'other_ticker')
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

- Lấy dữ liệu từ AlphaVantage

In [28]:
# Lấy dữ liệu các tỷ giá cần thiết từ Alpha Vantage
fx_pairs = [
    ('USD', 'VND', 'USD_VND'),
    ('USD', 'SEK', 'USD_SEK')
]

data_frames = []
for from_curr, to_curr, col_name in fx_pairs:
    df = get_data_from_av(from_curr, to_curr, col_name)
    if df is not None:
        data_frames.append(df)

av_ticker_df = pd.concat(data_frames, axis=1)
av_ticker_df = av_ticker_df.fillna(method='ffill').dropna()

### Phần bối cảnh vĩ mô

#### Dữ liệu các chỉ số tài chính

- Tính toán DXY từ các cặp tiền

In [29]:
currency_config = {
    'EUR_USD': {'source': 'other_ticker_df', 'weight': -0.576},
    'USD_JPY': {'source': 'other_ticker_df', 'weight': 0.136},
    'GBP_USD': {'source': 'other_ticker_df', 'weight': -0.119},
    'USD_CAD': {'source': 'other_ticker_df', 'weight': 0.091},
    'USD_CHF': {'source': 'other_ticker_df', 'weight': 0.036},
    'USD_SEK': {'source': 'av_ticker_df', 'weight': 0.042}
}

# Tạo DataFrame cho mỗi cặp tiền tệ
currency_data = {}
for pair, config in currency_config.items():
    if config['source'] == 'other_ticker_df':
        df = other_ticker_df[other_ticker_df['ticker'] == pair][['date', 'close']].set_index('date')
        df.columns = [pair]
    else:
        df = av_ticker_df[['USD_SEK']].copy()
        df.columns = [pair]
    
    currency_data[pair] = df

# Kết hợp tất cả dữ liệu tỷ giá
dxy_calculation_df = pd.concat(currency_data.values(), axis=1, sort=True)
dxy_calculation_df = dxy_calculation_df.sort_index().bfill().ffill()

# Tính toán chỉ số DXY theo công thức chuẩn
dxy_base = 50.14348112
dxy_calculation = dxy_base
for pair, config in currency_config.items():
    dxy_calculation *= (dxy_calculation_df[pair] ** config['weight'])

dxy_calculation_df['ticker'] = 'DXY'
dxy_calculation_df['close'] = dxy_calculation
dxy_calculation_df = dxy_calculation_df[['ticker', 'close']].reset_index()

# usd_vnd_df = av_ticker_df[['USD_VND']].rename(columns={'USD_VND': 'close'})
# usd_vnd_df['ticker'] = 'USD_VND'
# usd_vnd_df = usd_vnd_df.reset_index()

- Ghép tất cả vào ticker_dict

In [30]:
FINAL_DAYS = 5
TICKER_CONFIG = {
    # Chỉ số Việt Nam
    'VNINDEX':   {'df': full_index_df, 'market': 'hose', 'type': 'vn'},
    'VN30':      {'df': full_index_df, 'market': 'hose', 'type': 'vn'},
    'HNXINDEX':  {'df': full_index_df, 'market': 'hnx', 'type': 'vn'},
    'UPINDEX':   {'df': full_index_df, 'market': 'hnx', 'type': 'vn'},
    'VN30F1M':   {'df': full_index_df, 'market': 'derivatives', 'type': 'vn'},
    'VN30F2M':   {'df': full_index_df, 'market': 'derivatives', 'type': 'vn'},
    
    # Chỉ số quốc tế
    'DJI':       {'df': other_ticker_df, 'market': 'us', 'type': 'international'},
    'SPX':       {'df': other_ticker_df, 'market': 'us', 'type': 'international'},
    'FTSE':      {'df': other_ticker_df, 'market': 'eu', 'type': 'international'},
    'STOXX50E':  {'df': other_ticker_df, 'market': 'eu', 'type': 'international'},
    'N225':      {'df': other_ticker_df, 'market': 'asia', 'type': 'international'},
    'SSEC':      {'df': other_ticker_df, 'market': 'asia', 'type': 'international'},
}
FINAL_COLUMNS = ['date', 'ticker', 'close', 'cum_change', '1d_change', '5d_change', '20d_change', 'market', 'type']


# 2. XỬ LÝ THEO LUỒNG MỚI
final_df_list = []
for ticker, config in TICKER_CONFIG.items():
    # Lấy và sắp xếp dữ liệu nguồn
    source_df = config['df']
    temp_df = source_df[source_df['ticker'] == ticker].sort_values('date', ascending=False).copy()

    # TÍNH TOÁN TRƯỚC trên toàn bộ chuỗi dữ liệu đã sắp xếp để đảm bảo chính xác
    temp_df['1d_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
    temp_df['5d_change'] = temp_df['close'][::-1].pct_change(5)[::-1].fillna(0)
    temp_df['20d_change'] = temp_df['close'][::-1].pct_change(20)[::-1].fillna(0)
    
    # Gán các thông tin phân loại
    temp_df['market'] = config['market']
    temp_df['type'] = config['type']

    # CẮT 1 LẦN DUY NHẤT về 5 ngày sau khi đã tính toán xong
    final_rows = temp_df.head(FINAL_DAYS)

    # Tính toán cuối cùng trên 5 dòng đã cắt
    final_rows['cum_change'] = final_rows['1d_change'][::-1].cumsum()[::-1]
    final_df_list.append(final_rows)

# 3. KẾT QUẢ CUỐI CÙNG
weekly_history_data_df = pd.concat(final_df_list, ignore_index=True)[FINAL_COLUMNS]

#### Dữ liệu thị trường tiền tệ

In [31]:
# Lấy khung thời gian biểu đồ
chart_range = date_series['date'].iloc[:25]

# Đọc tất cả các file FiinProX từ thư mục Downloads
fiinprox_files = glob.glob(os.path.join(str(Path.home() / "Downloads"), "*FiinProX*.xlsx"))

# Lọc lại các file chứa dữ liệu thị trường mở và có số cột > 6
value_omo_fiinprox_files = []
rate_omo_fiinprox_files = []
rate_nhnn_finprox_files = []
for file in fiinprox_files:
    df = pd.read_excel(file)
    omo_name_check = df.applymap(lambda x: isinstance(x, str) and "Nghiệp vụ thị trường mở" in x).any().any()
    rate_name_check = df.applymap(lambda x: isinstance(x, str) and "Lãi suất thống kê của NHNN" in x).any().any()
    if omo_name_check and len(df.columns) > 6:
        value_omo_fiinprox_files.append(file)
    elif omo_name_check and len(df.columns) == 6:
        rate_omo_fiinprox_files.append(file)
    elif rate_name_check:
        rate_nhnn_finprox_files.append(file)
if len(value_omo_fiinprox_files) == 0:
    raise FileNotFoundError("❌ Chưa tải dữ liệu giá trị OMO từ FiinProX.")
elif len(rate_omo_fiinprox_files) == 0:
    raise FileNotFoundError("❌ Chưa tải dữ liệu lãi suất OMO từ FiinProX.")   
elif len(rate_nhnn_finprox_files) == 0:
    raise FileNotFoundError("❌ Chưa tải dữ liệu lãi suất thống kê của NHNN từ FiinProX.")

##### Bảng dữ liệu thị trường tiền tệ

In [32]:
# Sắp xếp fiinprox_files theo thời gian tạo (tăng dần) và lấy file mới nhất
last_value_omo_fiinprox_file = sorted(value_omo_fiinprox_files, key=os.path.getctime)[-1]
value_omo_df = pd.read_excel(last_value_omo_fiinprox_file, skiprows=4)

#Đọc dữ liệu daily OMO từ file mới nhất
daily_omo_df = value_omo_df.iloc[[1, 2, 5, 6]].reset_index(drop=True)
daily_omo_df = daily_omo_df.transpose().iloc[1:]
daily_omo_df.columns = ['tin_phieu_phat_hanh', 'tin_phieu_dao_han', 'reverse_repo_phat_hanh', 'reverse_repo_dao_han']
daily_omo_df.index = pd.to_datetime(daily_omo_df.index)
daily_omo_df = daily_omo_df[daily_omo_df.index.isin(chart_range)]
daily_omo_df = daily_omo_df.sort_index(ascending=False).fillna(0)
daily_omo_df['omo_bom_tien'] = daily_omo_df['tin_phieu_dao_han'] + daily_omo_df['reverse_repo_phat_hanh']
daily_omo_df['omo_hut_tien'] = daily_omo_df['tin_phieu_phat_hanh'] + daily_omo_df['reverse_repo_dao_han']
daily_omo_df['omo_bom_hut_rong'] = daily_omo_df['omo_bom_tien'] + daily_omo_df['omo_hut_tien']
daily_omo_df = daily_omo_df.iloc[:, [4, 5, 6]]

#Đọc dữ liệu outstanding OMO từ file mới nhất
outstanding_omo_df = value_omo_df.iloc[[3, 7]].reset_index(drop=True)
outstanding_omo_df = outstanding_omo_df.transpose().iloc[1:].fillna(0)
outstanding_omo_df.columns = ['tin_phieu_luu_hanh', 'reverse_repo_luu_hanh']
outstanding_omo_df.index = pd.to_datetime(outstanding_omo_df.index)
outstanding_omo_df['outstanding_omo_value'] = outstanding_omo_df['tin_phieu_luu_hanh'] + outstanding_omo_df['reverse_repo_luu_hanh']
outstanding_omo_df = outstanding_omo_df.sort_index(ascending=False)
outstanding_omo_df = outstanding_omo_df[outstanding_omo_df.index.isin(chart_range)]

#Đọc dữ liệu LSLNH từ file mới nhất
lslnh_df = pd.read_excel(last_value_omo_fiinprox_file, skiprows=4)
lslnh_df = lslnh_df.iloc[[16, 19, 20]].reset_index(drop=True)
lslnh_df = lslnh_df.transpose().iloc[1:]
lslnh_df.columns = ['lslnh_on', 'lslnh_1m', 'lslnh_3m']
lslnh_df.index = pd.to_datetime(lslnh_df.index)
lslnh_df = lslnh_df.sort_index(ascending=False)
lslnh_df = lslnh_df[lslnh_df.index.isin(chart_range)]

#Đọc dữ liệu tỉ giá từ Vietnambiz
for _ in range (5):
    try:
        dhtg_df = fetch_wichart_data(wichart_api_url_dict['tien_te'] + 'dhtg').set_index('date')
        dhtg_df.columns = ['dhtg_trung_tam', 'dhtg_tran', 'dhtg_san', 'dhtg_nhtm_ban_ra', 'dhtg_tu_do_ban_ra']
        dhtg_df = dhtg_df[['dhtg_trung_tam', 'dhtg_tran', 'dhtg_nhtm_ban_ra', 'dhtg_san', 'dhtg_tu_do_ban_ra']]
        dhtg_df.index = pd.to_datetime(dhtg_df.index)
        dhtg_df = dhtg_df[dhtg_df.index.isin(chart_range)]
        break
    except Exception as e:
        raise Exception(f"Lỗi khi lấy dữ liệu từ Wichart: {e}")
    
# Kết hợp tất cả các bảng dữ liệu lãi suất và tiền tệ
weekly_mm_data_df = pd.concat([lslnh_df, dhtg_df, daily_omo_df, outstanding_omo_df], axis=1, join='outer')
weekly_mm_data_df = weekly_mm_data_df.sort_index(ascending=False).bfill().reset_index().rename(columns={'index': 'date'})

##### Bảng dữ liệu chuyên về omo ngắn hạn

In [33]:
# Khởi tạo dictionary lưu file OMO
bill_omo_files = {}
rp_omo_files = {}

# Đọc dữ liệu lãi suất OMO từ các file FiinProX
temp_rate_omo_dict = {}
for file in rate_omo_fiinprox_files:
    df = pd.read_excel(file, skiprows=4).dropna()
    df = df.iloc[:, [0, 3, 4, 5]]
    df.columns = ['date', 'value', 'rate', 'term']
    df['date'] = pd.to_datetime(df['date'])
    df['term'] = df['term'].str.replace(' ngày', '').astype(int)
    temp_rate_omo_dict[file] = df

# Tìm ngày cuối cùng chung của các file OMO
omo_last_date_list = []
for df in temp_rate_omo_dict.values():
    omo_last_date_list.append(df['date'].max())
omo_last_date = min(omo_last_date_list)

# Tính trung bình kỳ hạn cho từng file OMO
omo_mean_term_dict = {}
for file, df in temp_rate_omo_dict.items():
    temp_df = df[df['date'] <= omo_last_date]
    omo_mean_term_dict[file] = temp_df['term'].mean()

# Phân loại file OMO thành bill và repo
bill_omo_files, repo_omo_files = clasify_omo_rate_df(omo_mean_term_dict)

# Lấy file bill và repo mới nhất
last_bill_omo_file = sorted(bill_omo_files, key=os.path.getctime)[-1]
last_repo_omo_file = sorted(repo_omo_files, key=os.path.getctime)[-1]

# Khởi tạo DataFrame lưu dữ liệu đáo hạn và phát hành OMO
issue_omo_df = pd.DataFrame()
for name, file in zip(['bill', 'repo'], [last_bill_omo_file, last_repo_omo_file]):
    df = pd.read_excel(file, skiprows=4).dropna()
    df = df.iloc[:, [0, 3, 4, 5]]
    df.columns = ['date', 'value', 'rate', 'term']
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = df['value'].apply(lambda x: -x if name == 'repo' else x)
    df['term'] = df['term'].str.replace(' ngày', '').astype(int)
    df['term'] = df['term'].apply(lambda x: 7 if x < 10 else x)
    df['type'] = name
    issue_omo_df = pd.concat([issue_omo_df, df], ignore_index=True)

# Chuẩn hóa dữ liệu phát hành OMO
weekly_omo_data_df = transform_rate_omo_data(issue_omo_df)
weekly_omo_data_df = weekly_omo_data_df.reset_index().rename(columns={'index': 'name'})

##### Bảng dữ liệu cho NHTM

In [34]:
last_rate_nhnn_finprox_files = sorted(rate_nhnn_finprox_files, key=os.path.getctime)[-1]
rate_nhnn_df = pd.read_excel(last_rate_nhnn_finprox_files, skiprows=4)
rate_nhnn_df = rate_nhnn_df.iloc[[1, 2, 6, 7]].transpose().iloc[1:]
rate_nhnn_df.columns = ['lshd_san', 'lshd_tran', 'lscv_san', 'lscv_tran']
rate_nhnn_df['lshd_tb'] = (rate_nhnn_df['lshd_san'] + rate_nhnn_df['lshd_tran']) / 2
rate_nhnn_df['lscv_tb'] = (rate_nhnn_df['lscv_san'] + rate_nhnn_df['lscv_tran']) / 2
rate_nhnn_df.index = pd.to_datetime(rate_nhnn_df.index)
rate_nhnn_df = rate_nhnn_df[['lshd_tran', 'lscv_tran', 'lshd_tb', 'lscv_tb', 'lshd_san', 'lscv_san']]

dtnh_df = fetch_wichart_data(wichart_api_url_dict['tien_te'] + 'dtnh').set_index('date')
dtnh_df.index = pd.to_datetime(dtnh_df.index)
dtnh_df.index += pd.Timedelta(days=10)
dtnh_df.index = dtnh_df.index.to_period('M').to_timestamp()
dtnh_df.columns = ['dtnh']

# Fill thêm dữ liệu vào các tháng chưa có
new_index = pd.date_range(rate_nhnn_df.index.min(), date_series['date'].max(), freq='MS').sort_values()
rate_nhnn_df_reindexed = rate_nhnn_df.reindex(new_index)
dtnh_df_reindexed = dtnh_df.reindex(new_index)
weekly_nhtm_data_df = pd.concat([rate_nhnn_df_reindexed, dtnh_df_reindexed], axis=1).ffill().sort_index(ascending=False)
weekly_nhtm_data_df = weekly_nhtm_data_df.reset_index().rename(columns={'index': 'date'})

#### Dữ liệu khác và hàng hoá

In [35]:
# DỮ liệu lấy từ Wichart API
other_ticker_congfig_1 = {
    'heo_hoi': {'type': 'Thực phẩm', 'name': 'Thị heo hơi', 'unit': 'VNĐ/kg'},
    'duong': {'type': 'Thực phẩm', 'name': 'Đường', 'unit': 'USD/tấn'},
    'soi_coton': {'type': 'Thực phẩm', 'name': 'Sợi cotton', 'unit': 'CNY/tấn'},
    'gao_tpxk': {'type': 'Thực phẩm', 'name': 'Gạo xuất khẩu', 'unit': 'VNĐ/tấn'},
    'quang_sat': {'type': 'Vật liệu xây dựng', 'name': 'Quặng sắt', 'unit': 'CNY/tấn'},
    'vang': {'type': 'Kim loại quý', 'name': 'Vàng trong nước', 'unit': 'Triệu VNĐ/lượng'},
    'hrc_trung_quoc': {'type': 'Vật liệu xây dựng', 'name': 'Thép HRC', 'unit': 'CNY/tấn'},
    'nhua_pvc_trung_quoc': {'type': 'Hóa chất', 'name': 'Hạt nhựa PVC', 'unit': 'CNY/tấn'},
    'phot_pho': {'type': 'Hóa chất', 'name': 'Phốt pho vàng', 'unit': 'CNY/tấn'},
    'ure_trung_dong': {'type': 'Hóa chất', 'name': 'Urê', 'unit': 'USD/tấn'},
    'cao_su_nhat_ban': {'type': 'Hóa chất', 'name': 'Cao su', 'unit': 'USD/tấn'}
}
other_data_df_list_1 = []
for ticker, config in other_ticker_congfig_1.items():
    for _ in range (5):
        try:
            temp_df = fetch_wichart_data(wichart_api_url_dict['hang_hoa'] + ticker)
            temp_df['date'] = pd.to_datetime(temp_df['date'])
            break
        except Exception as e:
            raise Exception(f"Lỗi khi lấy dữ liệu hàng hóa: {e}")
    if ticker == 'vang':
        temp_df = temp_df.iloc[:, [0, 2]]
    temp_df.columns = ['date', 'close']
    temp_df['1d_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
    temp_df['5d_change'] = temp_df['close'][::-1].pct_change(5)[::-1].fillna(0)
    temp_df['20d_change'] = temp_df['close'][::-1].pct_change(20)[::-1].fillna(0)
    temp_df = temp_df.iloc[[0]]
    if ticker == 'cao_su_nhat_ban':
        exchange_rate = other_ticker_df[other_ticker_df['ticker'] == 'USD_JPY']['close'].iloc[0]
        temp_df['close'] = temp_df['close']*1000 / exchange_rate
    temp_df['unit'] = config['unit']
    temp_df['type'] = config['type']
    temp_df['name'] = config['name']
    other_data_df_list_1.append(temp_df)
other_data_df_1 = pd.concat(other_data_df_list_1, ignore_index=True)

#Dữ liệu lấy từ Fireant
other_ticker_config_2 = {
    'XAU_USD':   {'df': other_ticker_df, 'type': 'Kinh loại quý', 'name': 'Vàng thế giới', 'unit': 'USD/ounce'},
    'DXY':       {'df': dxy_calculation_df, 'type': 'Tỷ giá', 'name': 'Chỉ số DXY', 'unit': 'Point'},
    'BTC_USD':   {'df': other_ticker_df, 'type': 'Tiền mã hoá', 'name': 'Bitcoin', 'unit': 'USD'},
    'ETH_USD':   {'df': other_ticker_df, 'type': 'Tiền mã hoá', 'name': 'Etherum', 'unit': 'USD'},
    'CLZ':       {'df': other_ticker_df, 'type': 'Năng lượng', 'name': 'Dầu WTI', 'unit': 'USD/thùng'},
    'NGZ':       {'df': other_ticker_df, 'type': 'Năng lượng', 'name': 'Khí tự nhiên', 'unit': 'USD/MMBtu'},
}
other_data_df_list_2 = []
for ticker, config in other_ticker_config_2.items():
    source_df = config['df']
    temp_df = source_df[source_df['ticker'] == ticker][['date', 'close']].sort_values('date', ascending=False).copy()
    # TÍNH TOÁN TRƯỚC trên toàn bộ chuỗi dữ liệu đã sắp xếp để đảm bảo chính xác
    temp_df['1d_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
    temp_df['5d_change'] = temp_df['close'][::-1].pct_change(5)[::-1].fillna(0)
    temp_df['20d_change'] = temp_df['close'][::-1].pct_change(20)[::-1].fillna(0)
    # Gán các thông tin phân loại
    temp_df['type'] = config['type']
    temp_df['name'] = config['name']
    temp_df['unit'] = config['unit']

    temp_df = temp_df.iloc[[0]]
    other_data_df_list_2.append(temp_df)
other_data_df_2 = pd.concat(other_data_df_list_2, ignore_index=True)

# Kết hợp dữ liệu hàng hóa và dữ liệu khác
weekly_other_data_df = pd.concat([other_data_df_1, other_data_df_2], ignore_index=True)
desired_order = [
    'Vàng thế giới', 'Vàng trong nước', 'Bitcoin', 'Etherum', 'Chỉ số DXY', 'Dầu WTI', 'Khí tự nhiên',
    'Quặng sắt', 'Thép HRC', 'Phốt pho vàng', 'Hạt nhựa PVC', 'Urê', 'Cao su', 
    'Sợi cotton', 'Đường', 'Thị heo hơi', 'Gạo xuất khẩu'
]
weekly_other_data_df['name'] = pd.Categorical(weekly_other_data_df['name'], categories=desired_order, ordered=True)
weekly_other_data_df = weekly_other_data_df.sort_values('name').reset_index(drop=True)

#### Dữ liệu vĩ mô (dự phòng)

In [36]:
# macro_configs = {
#     'gdp': {'rename_map': {'Tăng trưởng GDP': 'GDP'}, 'cols_to_keep': [0, 3], 'add_days': 0},
#     'cpi': {'rename_map': {'CPI': 'CPI'}, 'cols_to_keep': None, 'add_days': 10},
#     'iip': {'rename_map': {'Sản xuất công nghiệp': 'IIP'}, 'cols_to_keep': None, 'add_days': 10},
#     'pmi': {'rename_map': {'PMI': 'PMI'}, 'cols_to_keep': None, 'add_days': 10},
#     'hhdv': {'rename_map': {'Tổng mức bán lẻ HH và DV': 'tong_muc_ban_le_hhdv', 'Tăng trưởng': 'tang_truong_ban_le_hhdv'}, 'cols_to_keep': None, 'add_days': 10},
#     'cctm': {'rename_map': {'Xuất khẩu': 'xuat_khau', 'Nhập khẩu': 'nhap_khau', 'Cán cân thương mại': 'cctm'}, 'cols_to_keep': None, 'add_days': 10},
#     'dtnh': {'rename_map': {'Dự trữ ngoại hối': 'du_tru_ngoai_hoi'}, 'cols_to_keep': None, 'add_days': 10},
#     'ctt': {'rename_map': {'Cung tiền tệ': 'cung_tien_m2', 'Tăng trưởng': 'tang_truong_m2'}, 'cols_to_keep': None, 'add_days': 10},
#     'hd': {'rename_map': {'Tăng trưởng': 'tang_truong_tien_gui'}, 'cols_to_keep': [0, 2], 'add_days': 10},
#     'td': {'rename_map': {'Tăng trưởng': 'tang_truong_tin_dung'}, 'cols_to_keep': [0, 2], 'add_days': 10},
# }

# processed_dfs = []
# for endpoint, config in macro_configs.items():
#     for _ in range (5):
#         try:
#             df = fetch_wichart_data(wichart_api_url_dict['vi_mo'] + endpoint)
#             # Chỉ giữ lại các cột cần thiết (nếu có)
#             if config['cols_to_keep']:
#                 df = df.iloc[:, config['cols_to_keep']]
#             # Đổi tên cột và xử lý ngày tháng
#             df = df.rename(columns=config['rename_map'])
#             if endpoint == 'cpi':
#                 df['CPI'] = df['CPI'] / 100
#             df['date'] = pd.to_datetime(df['date'])
#             if config['add_days'] > 0:
#                 df['date'] += pd.Timedelta(days=config['add_days'])
#             df['date'] = df['date'].dt.strftime('%Y-%m')
#             # Đặt 'date' làm chỉ mục và thêm vào danh sách
#             processed_dfs.append(df.set_index('date'))
#             break
#         except Exception as e:
#             raise Exception(f"Lỗi khi lấy dữ liệu vĩ mô và tiền tệ: {e}")

# # Ghép nối, sắp xếp và lấy 13 dòng đầu tiên
# weekly_vimo_tiente_df = (pd.concat(processed_dfs, axis=1, join='outer').sort_index(ascending=False).reset_index().iloc[:25])
# weekly_vimo_tiente_df['date'] = pd.to_datetime(weekly_vimo_tiente_df['date'])
# weekly_vimo_tiente_df = weekly_vimo_tiente_df.sort_values('date')

#### Phần nhận xét các Index

In [37]:
# Thiết lập kết nối với gemini model
genai.configure(api_key=load_env("GEMINI_API"))
model_list = get_gemini_models()
standard_model_list = select_standard_models(model_list)
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

# Viết các đánh giá
vn_comment = generate_content_with_model_dict(standard_model_dict, weekly_data_comment_prompt(weekly_history_data_df, 'vn'), 'weekly_data_comment')
international_comment = generate_content_with_model_dict(standard_model_dict, weekly_data_comment_prompt(weekly_history_data_df, 'international'), 'weekly_data_comment')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm weekly_data_comment.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm weekly_data_comment.


### Phần phân tích thị trường

#### Dữ liệu VNINDEX và MS

- Tạo các bảng dữ liệu

In [38]:
ta_vn_index_df = full_index_df[full_index_df['ticker'] == 'VNINDEX'].reset_index(drop=True).iloc[:60]
ms_vn_index_df = full_ms_chart_df.iloc[:60]
weekly_ms_index_df = pd.merge(ta_vn_index_df[['date', 'close']], ms_vn_index_df, on='date', how='left')

- Vẽ biểu đồ kĩ thuật

In [39]:
vnindex_chart_df = full_index_df[full_index_df['ticker'] == 'VNINDEX'].iloc[:120].sort_values('date').reset_index(drop=True)
image_name = 'TA_WEEKLY_VNINDEX.png'

line_name_dict = {
    'SMA_20': 'SMA 20',
    'SMA_60': 'SMA 60',
    'week_open': 'WEEK OPEN',
    'month_open': 'MONTH OPEN',
    'quarter_open': 'QUARTER OPEN',
    'year_open': 'YEAR OPEN',
    'month_prev_high': 'LAST MHIGH',
    'month_prev_low': 'LAST MLOW',
    'quarter_prev_high': 'LAST QHIGH',
    'quarter_prev_low': 'LAST QLOW',
    'MFIBO_0382': 'MFIBO 0.382',
    'MFIBO_0500': 'MFIBO 0.500',
    'MFIBO_0618': 'MFIBO 0.618',
    'QFIBO_0382': 'QFIBO 0.382',
    'QFIBO_0500': 'QFIBO 0.500',
    'QFIBO_0618': 'QFIBO 0.618',
	'YFIBO_0382': 'YFIBO 0.382',
    'YFIBO_0500': 'YFIBO 0.500',
    'YFIBO_0618': 'YFIBO 0.618'
}

chart_config = create_chart_config(
	title_font_size=30,
	axis_font_size=24,
	tag_font_size=24,
	price_tag_font_size=24,
    min_spacing_ratio = 0.042,
	margin=dict(l=20, r=220, t=20, b=20, pad=10)
)

max_attempts = 5
for attempt in range(max_attempts):
    try:
        fig, fig_images = create_financial_chart(
            vnindex_chart_df, # DataFrame của bạn
            width=1400,
            height=1200,
            line_name_dict=line_name_dict,
            line_columns=list(line_name_dict.keys()),
            chart_config=chart_config,
            path='../output/weekly/',
            image_name = image_name
        )
        upload_to_r2(fig_images, image_name)
        break
    except Exception as e:
        if attempt == max_attempts - 1:
            raise e
# fig.show()

#### Nhận xét VNINDEX và MS

In [40]:
ta_vnindex_comment = generate_content_with_model_dict(standard_model_dict, weekly_vnindex_comment_prompt(ta_vn_index_df), 'weekly_vnindex_comment')
ms_chart_comment = generate_content_with_model_dict(standard_model_dict, weekly_ms_comment_prompt(ms_vn_index_df), 'weekly_ms_comment')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm weekly_vnindex_comment.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm weekly_ms_comment.


#### Khối ngoại và tự doanh

- Lịch sử mua bán ròng trong tuần

In [41]:
weekly_nntd_history_df = nntd_index_df[nntd_index_df['ticker'].isin(['VNINDEX','HNXINDEX', 'UPINDEX'])].drop(columns=['ticker'])
weekly_nntd_history_df = weekly_nntd_history_df.groupby(['date', 'type']).sum().sort_values('date', ascending=False).reset_index()
latest_dates = weekly_nntd_history_df['date'].drop_duplicates().sort_values(ascending=False).head(5)
weekly_nntd_history_df = weekly_nntd_history_df[weekly_nntd_history_df['date'].isin(latest_dates)]
weekly_nntd_history_df[[col for col in weekly_nntd_history_df.columns if '_volume' in col]] = \
    weekly_nntd_history_df[[col for col in weekly_nntd_history_df.columns if '_volume' in col]] / 1000000

- Top cổ phiếu mua bán ròng

In [42]:
temp_df_list = []

for stock_type in ['NN', 'TD']:
    type_df = pd.DataFrame()
    
    for ticker in nntd_stock_df['ticker'].unique():
        temp_df = nntd_stock_df[(nntd_stock_df['ticker'] == ticker) & (nntd_stock_df['type'] == stock_type)]
        temp_df = temp_df.sort_values('date', ascending=False).reset_index(drop=True)
        
        if len(temp_df) == 0:
            continue
            
        temp_df['weekly_value'] = temp_df['net_value'][::-1].rolling(window=5, min_periods=1).sum()[::-1]
        type_df = pd.concat([type_df, temp_df[['type', 'ticker', 'weekly_value']].iloc[[0]]], ignore_index=True)
    
    type_df = type_df.sort_values('weekly_value', ascending=False).reset_index(drop=True)
    temp_df_list.append(pd.concat([type_df.head(10), type_df.tail(10)], ignore_index=True))

# Gán kết quả và tổng hợp
weekly_nntd_stock_df = pd.concat(temp_df_list, ignore_index=True)
weekly_nntd_stock_df['top_check'] = (weekly_nntd_stock_df['weekly_value'] > 0).astype(int)
weekly_nntd_stock_df['weekly_value'] = abs(weekly_nntd_stock_df['weekly_value'])

#### Biến động vốn hoá

In [43]:
# --- PHẦN 1: TÍNH TOÁN NET_CAP CHO TỪNG CỔ PHIẾU ---
open_week_date = full_stock_df['date'].min()
close_week_date = full_stock_df['date'].max()

open_week_stock_cap_df = full_stock_df[full_stock_df['date'] == open_week_date][['ticker', 'cap']].rename(columns={'cap': 'open_cap'})
close_week_stock_cap_df = full_stock_df[full_stock_df['date'] == close_week_date][['ticker', 'cap']].rename(columns={'cap': 'close_cap'})

stock_cap_df = pd.merge(open_week_stock_cap_df, close_week_stock_cap_df, on='ticker', how='outer')
stock_cap_df.fillna({'open_cap': 0, 'close_cap': 0}, inplace=True)
stock_cap_df['net_cap'] = stock_cap_df['close_cap'] - stock_cap_df['open_cap']
stock_cap_df['industry'] = stock_cap_df['ticker'].map(full_stock_classification_df.set_index('ticker')['industry_name']).map(name_map_dict)
stock_cap_df = stock_cap_df.sort_values('net_cap', ascending=False).reset_index(drop=True)

# --- PHẦN 2: TÍNH TOÁN TỔNG LỰC KÉO/ĐẨY TOÀN THỊ TRƯỜNG ---
# Vẫn tính toán các giá trị này như biến thông thường để sử dụng
gainers_df = stock_cap_df[stock_cap_df['net_cap'] > 0]
losers_df = stock_cap_df[stock_cap_df['net_cap'] < 0]

cap_gain = gainers_df['net_cap'].sum()
cap_loss = abs(losers_df['net_cap'].sum()) # Dùng abs() để mẫu số luôn dương

# --- PHẦN 3: TỔNG HỢP DỮ LIỆU THỊ TRƯỜNG, NGÀNH, CỔ PHIẾU ---
# 3.1. Tổng hợp cho toàn thị trường
market_cap_sum = stock_cap_df.select_dtypes(include=[np.number]).sum(axis=0)
change_market_cap_df = pd.DataFrame([market_cap_sum])
change_market_cap_df['ticker'] = 'Thị trường'
change_market_cap_df['type'] = 'market'

# 3.2. Tổng hợp cho các ngành
change_industry_cap_df = (
    stock_cap_df
    .drop(columns=['ticker'])
    .groupby('industry').sum().reset_index()
    .rename(columns={'industry': 'ticker'})
    .sort_values('net_cap', ascending=False)
)
change_industry_cap_df['type'] = 'industry'

# 3.3. Tổng hợp cho top cổ phiếu
change_stock_cap_df = pd.concat([
    stock_cap_df.head(5).drop(columns=['industry']),
    stock_cap_df.tail(5).drop(columns=['industry'])],
    ignore_index=True
)
change_stock_cap_df['type'] = 'stock'

# 3.4. Ghép các bảng lại
market_cap_change_df = pd.concat([change_market_cap_df, change_industry_cap_df, change_stock_cap_df], ignore_index=True)

# --- PHẦN 4: TÍNH TOÁN CÁC CHỈ SỐ CUỐI CÙNG --- ## MODIFIED ##
# ## NEW ##: Thêm 2 cột mới để chứa giá trị tổng lực kéo và đẩy
# Các giá trị này sẽ được lặp lại trên tất cả các hàng
market_cap_change_df['cap_gain'] = cap_gain
market_cap_change_df['cap_loss'] = cap_loss

# Tính % thay đổi vốn hóa (giữ nguyên)
market_cap_change_df['change_cap'] = np.divide(
    (market_cap_change_df['close_cap'] - market_cap_change_df['open_cap']),
    market_cap_change_df['open_cap']
)
market_cap_change_df['change_cap'].replace([np.inf, -np.inf], 0, inplace=True)

# Tính toán tác động Kéo/Đẩy
market_cap_change_df['impact_group'] = 'Không đổi'
market_cap_change_df['impact_ratio'] = 0.0

# Áp dụng logic cho nhóm "positive"
gainers_mask = market_cap_change_df['net_cap'] > 0
market_cap_change_df.loc[gainers_mask, 'impact_group'] = 'positive'
# ## MODIFIED ##: Chia cho cột 'cap_gain' thay vì biến
market_cap_change_df.loc[gainers_mask, 'impact_ratio'] = market_cap_change_df.apply(lambda x: 
    (x['net_cap'] / x['cap_gain']) * (x['cap_gain'] / (x['cap_gain'] + x['cap_loss'])) , axis=1)

# Áp dụng logic cho nhóm "negative"
losers_mask = market_cap_change_df['net_cap'] < 0
market_cap_change_df.loc[losers_mask, 'impact_group'] = 'nagative' # "nagative" -> "negative"
# ## MODIFIED ##: Chia cho cột 'cap_loss' thay vì biến
market_cap_change_df.loc[losers_mask, 'impact_ratio'] = market_cap_change_df.apply(lambda x: 
    (x['net_cap'] / x['cap_loss']) * (x['cap_loss'] / (x['cap_gain'] + x['cap_loss'])) , axis=1)

# Xử lý riêng cho dòng 'Thị trường'
market_mask = market_cap_change_df['type'] == 'market'
market_cap_change_df.loc[market_mask, 'impact_group'] = 'neutral'
market_cap_change_df.loc[market_mask, 'impact_ratio'] = 1.0

# ## MODIFIED ##: Sắp xếp lại các cột cuối cùng, thêm 2 cột mới vào
market_cap_change_df = market_cap_change_df[[
    'type', 'ticker', 'open_cap', 'close_cap', 
    'cap_gain', 'cap_loss', 'net_cap', 'change_cap',
    'impact_group', 'impact_ratio'
]]

### Phần danh mục khuyến nghị

In [44]:
weekly_portfolio_df = pd.read_excel('../data/weekly/cts_portfolio.xlsx')
weekly_portfolio_df['industry'] = weekly_portfolio_df['ticker'].map(full_stock_classification_df.set_index('ticker')['industry_name']).map(name_map_dict)

t5_pct_change_list = []
for ticker in weekly_portfolio_df['ticker'].unique():
    temp_df = full_stock_df[full_stock_df['ticker'] == ticker]
    temp_df['t5_pct_change'] = temp_df['close'][::-1].pct_change(4).fillna(0)[::-1]
    t5_pct_change_list.append(temp_df['t5_pct_change'].iloc[0])

#Ghép các cột thông tin
ref_stock_df = origin_stock_df[origin_stock_df['date'] == max(origin_stock_df['date'])].set_index('ticker')
weekly_portfolio_df = weekly_portfolio_df.set_index('ticker')

# Các cột biến động
weekly_portfolio_df['current_price'] = ref_stock_df['close']
weekly_portfolio_df['t5_pct_change'] = t5_pct_change_list
weekly_portfolio_df['mean_vol'] = origin_stock_df.groupby('ticker')['vol_ratio'].mean()

# Các cột khoảng cách giá trị kỹ thuật
weekly_portfolio_df['distance_mopen'] = (ref_stock_df['close'] - ref_stock_df['month_open']) / ref_stock_df['month_open']
weekly_portfolio_df['distance_mhigh_prev'] = (ref_stock_df['close'] - ref_stock_df['month_prev_high']) / ref_stock_df['month_prev_high']
weekly_portfolio_df['distance_mlow_prev'] = (ref_stock_df['close'] - ref_stock_df['month_prev_low']) / ref_stock_df['month_prev_low']
weekly_portfolio_df['distance_yopen'] = (ref_stock_df['close'] - ref_stock_df['year_open']) / ref_stock_df['year_open']
weekly_portfolio_df['distance_yhigh_prev'] = (ref_stock_df['close'] - ref_stock_df['year_prev_high']) / ref_stock_df['year_prev_high']
weekly_portfolio_df['distance_ylow_prev'] = (ref_stock_df['close'] - ref_stock_df['year_prev_low']) / ref_stock_df['year_prev_low']

weekly_portfolio_df = weekly_portfolio_df.reset_index()

# Chuyển đổi các cột giá trị sang kiểu số và xử lý NaN/Inf
for col in weekly_portfolio_df.columns[2:]:
    weekly_portfolio_df[col] = pd.to_numeric(weekly_portfolio_df[col], errors='coerce')
    weekly_portfolio_df[col] = weekly_portfolio_df[col].replace([np.inf, -np.inf], np.nan)
    weekly_portfolio_df[col] = weekly_portfolio_df[col].round(6)

# Sắp xếp theo 'industry' trước, sau đó đến 't5_pct_change' giảm dần
weekly_portfolio_df = weekly_portfolio_df.sort_values(['industry', 't5_pct_change'], ascending=[True, False]).reset_index(drop=True)

### Lưu vào MSSQL


In [45]:
today = date_series['date'].max()
first_day_of_week = today - pd.Timedelta(days=today.weekday())

weekly_data_comments_df = pd.DataFrame(
    {
        "type":     ["vn", "international", "ta_vnindex", "ms_chart", "update_time", "date_range"],
        "group":    ["news", "news", "chart", "chart", "time", "date"],
        "comment": [
            vn_comment,
            international_comment,
            ta_vnindex_comment,
            ms_chart_comment,
            time_series["date"].max().strftime("%H:%M ngày %d/%m/%Y"),
            f"NGÀY {first_day_of_week.strftime('%d/%m/%Y')} - {today.strftime('%d/%m/%Y')}"
        ],
    }
)

In [46]:
%%capture
save_to_mssql(cts_engine, weekly_history_data_df, 'weekly_history_data')
save_to_mssql(cts_engine, weekly_nntd_history_df, 'weekly_nntd_history')
save_to_mssql(cts_engine, weekly_nntd_stock_df, 'weekly_nntd_stock')
save_to_mssql(cts_engine, weekly_ms_index_df, 'weekly_ms_index')
save_to_mssql(cts_engine, weekly_data_comments_df, 'weekly_data_comments')
save_to_mssql(cts_engine, market_cap_change_df, 'weekly_market_cap')
save_to_mssql(cts_engine, weekly_portfolio_df, 'weekly_stock_portfolio')
save_to_mssql(cts_engine, weekly_other_data_df, 'weekly_other_data')
save_to_mssql(cts_engine, weekly_omo_data_df, 'weekly_omo_data')
save_to_mssql(cts_engine, weekly_mm_data_df, 'weekly_mm_data')
save_to_mssql(cts_engine, weekly_nhtm_data_df, 'weekly_nhtm_data')